In [1]:
import pymc
import numpy as np
from scipy.optimize import root_scalar
from scipy.integrate import solve_ivp
import astropy.coordinates as coord
import astropy.units as u
import corner
import matplotlib.pyplot as plt
from scipy.stats import truncnorm
import multiprocessing
from functools import partial

In [25]:
# 常量
G = 43018  # 引力常数，单位：kpc (km/s)^2 / 10^10 Msun
obs_ra = 10.68
obs_ra_err = 0.1
obs_dec = 41.27
obs_dec_err = 0.1
obs_distance = 761
obs_distance_err = 11
obs_pmra = 46.9*1e-3
obs_pmra_err = 10*1e-3
obs_pmdec = -29.1*1e-3
obs_pmdec_err = 10*1e-3
obs_vr = -301
obs_vr_err = 1

data = [obs_ra, obs_dec, obs_distance, obs_pmra, obs_pmdec, obs_vr]
obs_err = [obs_ra_err, obs_dec_err, obs_distance_err, obs_pmra_err, obs_pmdec_err, obs_vr_err]


def rotation_matrix_x(theta):
    """
    绕 x 轴旋转的旋转矩阵 (角度 theta)
    参数:
        theta: 绕 x 轴旋转的角度 (弧度)
    返回:
        3x3 旋转矩阵
    """
    return np.array([
        [1, 0, 0],
        [0, np.cos(theta), -np.sin(theta)],
        [0, np.sin(theta), np.cos(theta)]
    ])
    
def rotation_matrix_y(phi):
    """
    绕 y 轴旋转的旋转矩阵 (角度 phi)
    参数:
        phi: 绕 y 轴旋转的角度 (弧度)
    返回:
        3x3 旋转矩阵
    """
    return np.array([
        [np.cos(phi), 0, np.sin(phi)],
        [0, 1, 0],
        [-np.sin(phi), 0, np.cos(phi)]
    ])

def rotation_matrix_z(phi):
    """
    绕 z 轴旋转的旋转矩阵 (角度 phi)
    参数:
        phi: 绕 z 轴旋转的角度 (弧度)
    返回:
        3x3 旋转矩阵
    """
    return np.array([
        [np.cos(phi), -np.sin(phi), 0],
        [np.sin(phi), np.cos(phi), 0],
        [0, 0, 1]
    ])

#坐标系转换
def Galactic_to_icrs(pos,vel):
    gc = coord.SkyCoord(x= pos[0]* u.kpc, y=pos[1]* u.kpc,
            z= pos[2]* u.kpc,
            v_x= vel[0]* u.km / u.s,
            v_y= vel[1]* u.km / u.s,
            v_z= vel[2]* u.km / u.s, frame='galactocentric',
            representation_type='cartesian', differential_type='cartesian',
            galcen_distance=8.122* u.kpc, z_sun=20.8 * u.pc,
            galcen_v_sun=coord.CartesianDifferential(
                 [12.9, 245.6, 7.8] * u.km / u.s))
    c = gc.transform_to(coord.ICRS)
    ra = np.array(c.ra)
    dec = np.array(c.dec)
    distance = np.array(c.distance)
    pmra = np.array(c.pm_ra_cosdec)
    pmdec = np.array(c.pm_dec)
    vr = np.array(c.radial_velocity)
    return ra, dec, distance, pmra, pmdec, vr

# 定义二体问题的微分方程
def two_body_equation(t, y, M):
    """
    二体问题的微分方程
    参数:
        t: 时间 (Gyr)
        y: 状态向量 [x, y, z, vx, vy, vz]
        M: 中心天体质量 (10^10 Msun)
    返回:
        dydt: 状态向量的导数 [vx, vy, vz, ax, ay, az]
    """
    x, y, z, vx, vy, vz = y
    r = np.sqrt(x**2 + y**2 + z**2)
    ax = -G * M * x / r**3
    ay = -G * M * y / r**3
    az = -G * M * z / r**3
    return [vx, vy, vz, ax, ay, az]

# 定义运动学模型
def orbit_model(params):
    """
    根据轨道参数计算 r, v_rad, v_tan
    参数:
        params: [a, e, M, t_peri]，分别为半长轴、偏心率、总质量、近日点时间
        t_obs: 当前时间 (Gyr)
    返回:
        r: 距离 (kpc)
        v_rad: 径向速度 (km/s)
        v_tan: 切向速度 (km/s)
    """
    a, e, M, theta, phi, alpha, T_uni = params

    
    # 微分方程的初始条件设置
    t_span = (0, T_uni)
    r0 = a * (1 - e )
    v_factor = np.sqrt(G * M / a)
    v_tan = v_factor * np.sqrt(1 - e**2) / (1 - e )
    # 计算坐标
    rot_matrix = np.dot(rotation_matrix_y(theta), np.dot(rotation_matrix_z(phi), rotation_matrix_x(alpha)))
    pos = np.dot(rot_matrix, np.array([r0, 0, 0]))
    vel = np.dot(rot_matrix, np.array([0, v_tan, 0]))
    
    
    #求解方程
    y0 = np.concatenate((pos, vel))

    t_eval = np.linspace(t_span[0], t_span[1], 1000)  # 时间点

    # 数值积分
    solution = solve_ivp(two_body_equation, t_span, y0, args=(M,), t_eval=t_eval, rtol=1e-6, atol=1e-6, method='RK45')
    
    # 提取 t = T_uni 时的解
    final_pos = solution.y[:3, -1]  # 最后一列是位置 [x, y, z]
    final_vel = solution.y[3:, -1]  # 最后一列是速度 [vx, vy, vz]

    # 转换为 ICRS 坐标
    ra, dec, distance, pmra, pmdec, vr = Galactic_to_icrs(final_pos, final_vel)
    
    return ra, dec, distance, pmra, pmdec, vr


def log_likelihood(params):
    """
    计算给定参数的对数似然值
    参数:
        params: 模型参数 [a, e, M, theta, phi, alpha, T_uni]
        data: 观测数据
        obs_err: 观测误差
    返回:
        对数似然值
    """
    # 解包参数
    a, e, M, theta, phi, alpha, T_uni = params

    # 计算轨道周期 T
    T = np.sqrt(a**3 / (G * M)) * 2 * np.pi
    if T <= 10:
        return -1e30  # 返回一个非常小的似然值

    # 解包观测数据和误差
    obs_ra, obs_dec, obs_distance, obs_pmra, obs_pmdec, obs_vr = data
    obs_ra_err, obs_dec_err, obs_distance_err, obs_pmra_err, obs_pmdec_err, obs_vr_err = obs_err

    # 计算模型值
    model_ra, model_dec, model_distance, model_pmra, model_pmdec, model_vr = orbit_model(params)

    # 计算似然
    chi2 = (model_ra - obs_ra)**2 / obs_ra_err**2 + \
        (model_dec - obs_dec)**2 / obs_dec_err**2 + \
        (model_distance - obs_distance)**2 / obs_distance_err**2 + \
        (model_pmra - obs_pmra)**2 / obs_pmra_err**2 + \
        (model_pmdec - obs_pmdec)**2 / obs_pmdec_err**2 + \
        (model_vr - obs_vr)**2 / obs_vr_err**2

    return -0.5 * chi2



# 定义参数的先验变换函数
def prior_transform(u):
    """
    将单位立方体中的参数 u 转换为物理参数空间中的参数
    参数:
        u: 单位立方体中的参数 (0 <= u_i <= 1)
    返回:
        params: 物理参数空间中的参数
    """
    a = truncnorm.ppf(u[0], -2, 2, loc=500, scale=100)  # a ~ N(500, 100)
    e = 1 - np.exp(-10 * u[1])      # e ~ U(0, 1)
    M = truncnorm.ppf(u[2], -1, 1, loc=400, scale=300)  # M ~ N(400, 300)
    phi = -np.pi/2*u[3] # phi ~ U(-pi/2,0)
    theta = -np.pi * u[4] # theta ~ U(-pi,0)
    alpha = np.pi * u[5] * 2 
    T_uni = truncnorm.ppf(u[6], -2, 2, loc=13.8, scale=0.1)
    T = np.sqrt(a**3 / (G * M))*2 * np.pi


    return [a, e, M, theta, phi, alpha, T_uni]



In [ ]:



ndim = 7  # 参数维度
ncores = 10

with multiprocessing.Pool(ncores) as pool:

    sampler = dynesty.NestedSampler(log_likelihood, prior_transform, ndim, nlive=1000, bound='multi', sample="auto", pool=pool,
                                    queue_size=10)
    sampler.run_nested(print_progress=True)
results = sampler.results

3933it [15:52,  1.03s/it, bound: 11 | nc: 12 | ncall: 51038 | eff(%):  7.706 | loglstar:   -inf < -10423.792 <    inf | logz: -10435.326 +/-    nan | dlogz: 10321.150 >  1.009] 

In [ ]:
from dynesty import plotting as dyplot
dyplot.cornerplot(results)

LinAlgError: The data appears to lie in a lower-dimensional subspace of the space in which it is expressed. This has resulted in a singular data covariance matrix, which cannot be treated using the algorithms implemented in `gaussian_kde`. Consider performing principle component analysis / dimensionality reduction and using `gaussian_kde` with the transformed data.